In [3]:
import torch
from mh_nlp.application.use_cases.train_model import TrainModelUseCase

# Détection du hardware
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Utilisation de : {device}")

# On suppose que 'splits' contient déjà {"train": DTO, "val": DTO, "test": DTO} 
# issus de l'étape précédente (SplitDatasetUseCase)

Utilisation de : cpu


In [4]:
from loguru import logger
from mh_nlp.domain.services.text_cleaner import TextCleaner
from mh_nlp.infrastructure.data.kaggle_repository import KaggleDatasetRepository
from mh_nlp.application.use_cases.build_dataset import BuildCleanDatasetUseCase

# 1.
label_mapping = {
    "Anxiety": 0, 
    "Normal": 1, 
    "Depression": 2,
}

# 2. Initialisation de l'accès aux données et du service de nettoyage
repository = KaggleDatasetRepository(
    csv_path="../data/raw/mental_health.csv",
    label_mapping=label_mapping,
)

cleaner = TextCleaner()

# 3. Injection des dépendances dans le cas d'usage
use_case = BuildCleanDatasetUseCase(
    repository=repository, 
    cleaner=cleaner
)

# 4. Lancement du pipeline et récupération du DatasetDTO
dataset_final = use_case.execute()

2026-01-17 15:14:15.591 | INFO     | mh_nlp.application.use_cases.build_dataset:execute:47 - Démarrage du pipeline BuildCleanDataset.
2026-01-17 15:14:15.591 | INFO     | mh_nlp.infrastructure.data.kaggle_repository:load:33 - Démarrage du chargement des données depuis : ../data/raw/mental_health.csv
2026-01-17 15:14:16.049 | SUCCESS  | mh_nlp.infrastructure.data.kaggle_repository:load:57 - Infrastructure : 1000 paires (Document, Label) créées.
2026-01-17 15:14:16.054 | DEBUG    | mh_nlp.application.use_cases.build_dataset:execute:66 - Traitement unitaire de 1000 documents.
2026-01-17 15:14:16.066 | SUCCESS  | mh_nlp.application.use_cases.build_dataset:execute:92 - Pipeline terminé : 1000 entités synchronisées. (0 supprimées car vides).


In [5]:
from mh_nlp.application.use_cases.split_dataset import SplitDatasetUseCase

# 1. Instanciation du Use Case avec vos ratios
# Ici: 70% Train / 10% Val / 20% Test
splitter = SplitDatasetUseCase(test_size=0.2, val_size=0.1)

# 2. Exécution du split
splits = splitter.execute(dataset_final)

# 3. Récupération des résultats
train_set = splits["train"]
val_set = splits["val"]
test_set = splits["test"]

print(f"Taille Train : {train_set.total_records}")
print(f"Premier document Train : {train_set.documents[0].text}")
print(f"Premier label Train : {train_set.labels[0]}")

2026-01-17 15:14:19.865 | INFO     | mh_nlp.application.use_cases.split_dataset:execute:53 - Découpage du dataset (Total: 1000) | Cible: Test=0.2, Val=0.1
2026-01-17 15:14:19.871 | SUCCESS  | mh_nlp.application.use_cases.split_dataset:execute:86 - Split réussi : Train=700 | Val=100 | Test=200


Taille Train : 700
Premier document Train : anxiety fear overthinking at the same time can be called panic attack right
Premier label Train : 0


In [7]:
from mh_nlp.infrastructure.nlp.hf_tokenizer import HuggingFaceTokenizer
from mh_nlp.infrastructure.models.distilbert_classifier import DistilBertClassifier

# 1. Initialisation
tokenizer_db = HuggingFaceTokenizer("distilbert-base-uncased", max_length=64)
model_db = DistilBertClassifier(
    model_name="distilbert-base-uncased", 
    num_labels=len(label_mapping), 
    tokenizer=tokenizer_db, 
    device=device
)

# 2. Exécution (Le Use Case reste agnostique du modèle)
train_use_case = TrainModelUseCase(model_db)
train_use_case.execute(splits["train"], splits["val"])

2026-01-17 15:17:40.480 | INFO     | mh_nlp.infrastructure.nlp.hf_tokenizer:__init__:34 - Chargement du tokenizer HF : distilbert-base-uncased (max_length=64)
2026-01-17 15:17:41.108 | SUCCESS  | mh_nlp.infrastructure.nlp.hf_tokenizer:__init__:36 - Tokenizer distilbert-base-uncased chargé avec succès.
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2026-01-17 15:17:41.370 | DEBUG    | mh_nlp.application.use_cases.train_model:__init__:28 - TrainModelUseCase initialisé avec DistilBertClassifier
2026-01-17 15:17:41.416 | INFO     | mh_nlp.application.use_cases.train_model:execute:50 - Démarrage de l'entraînement : 700 docs (train) | 100 docs (val)
2026-01-17 15:17:41.417 | DEBUG    | mh_nl

In [8]:
from mh_nlp.infrastructure.models.distilbert_cnn_classifier import HybridClassifier

# On réutilise le même tokenizer que DistilBERT
model_hybrid = HybridClassifier(
    model_name="distilbert-base-uncased", 
    num_labels=3, 
    tokenizer=tokenizer_db, 
    device=device
)

train_use_case_hybrid = TrainModelUseCase(model_hybrid)
train_use_case_hybrid.execute(splits["train"], splits["val"])

2026-01-18 05:13:08.798 | DEBUG    | mh_nlp.application.use_cases.train_model:__init__:28 - TrainModelUseCase initialisé avec HybridClassifier
2026-01-18 05:13:08.804 | INFO     | mh_nlp.application.use_cases.train_model:execute:50 - Démarrage de l'entraînement : 700 docs (train) | 100 docs (val)
2026-01-18 05:13:08.808 | DEBUG    | mh_nlp.infrastructure.nlp.hf_tokenizer:tokenize:63 - Tokenisation en cours pour 700 documents (Modèle: distilbert-base-uncased)
2026-01-18 05:13:08.903 | DEBUG    | mh_nlp.infrastructure.nlp.hf_tokenizer:tokenize:76 - Tokenisation terminée. Forme du tenseur : [700, 64]
2026-01-18 05:13:08.911 | INFO     | mh_nlp.infrastructure.training.torch_trainer:__init__:18 - Trainer initialisé sur cpu
Epoch 1/3: 100%|██████████| 44/44 [01:11<00:00,  1.62s/it, loss=0.3802]
2026-01-18 05:14:20.015 | INFO     | mh_nlp.infrastructure.training.torch_trainer:train:53 - Epoch 1 | Train Loss: 0.6513
Epoch 2/3: 100%|██████████| 44/44 [00:46<00:00,  1.05s/it, loss=0.0488]
2026-0

In [9]:
from mh_nlp.infrastructure.models.roberta_classifier import RobertaClassifier

# Attention : RoBERTa a son propre tokenizer
tokenizer_roberta = HuggingFaceTokenizer("roberta-base", max_length=64)
model_roberta = RobertaClassifier(
    model_name="roberta-base", 
    num_labels=3, 
    tokenizer=tokenizer_roberta, 
    device=device
)

train_use_case_roberta = TrainModelUseCase(model_roberta)
train_use_case_roberta.execute(splits["train"], splits["val"])

2026-01-18 05:16:33.799 | INFO     | mh_nlp.infrastructure.nlp.hf_tokenizer:__init__:34 - Chargement du tokenizer HF : roberta-base (max_length=64)
2026-01-18 05:16:34.542 | SUCCESS  | mh_nlp.infrastructure.nlp.hf_tokenizer:__init__:36 - Tokenizer roberta-base chargé avec succès.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2026-01-18 05:16:34.885 | DEBUG    | mh_nlp.application.use_cases.train_model:__init__:28 - TrainModelUseCase initialisé avec RobertaClassifier
2026-01-18 05:16:34.886 | INFO     | mh_nlp.application.use_cases.train_model:execute:50 - Démarrage de l'entraînement : 700 docs (train) | 100 docs (val)
2026-01-18 05:16:34.887 | DEBUG    | mh_nlp.infrastructure.

In [11]:
from mh_nlp.infrastructure.nlp.keras_tokenizer import KerasTextTokenizer
from mh_nlp.infrastructure.models.cnn_classifier import KerasCNNClassifier

# 1. Tokenizer spécifique à Keras
k_tokenizer = KerasTextTokenizer(vocab_size=5000, max_length=100)

# 2. Modèle Keras (implémente aussi TextClassifier)
model_keras = KerasCNNClassifier(
    vocab_size=5000,
    embedding_dim=128,
    max_length=100,
    num_labels=len(label_mapping),
    tokenizer=k_tokenizer
)

# 3. La magie de l'interface : le Use Case ne voit pas la différence avec PyTorch
train_use_case_keras = TrainModelUseCase(model_keras)
train_use_case_keras.execute(splits["train"], splits["val"])

2026-01-18 05:31:33.110 | INFO     | mh_nlp.infrastructure.nlp.keras_tokenizer:__init__:32 - KerasTextTokenizer initialisé (Vocab target: 5000, Max length: 100)
/Users/surelmanda/Downloads/ml-projects/Clean-Architecture-MLops/mental-health-nlp/.venv/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:100: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
2026-01-18 05:31:33.460 | DEBUG    | mh_nlp.application.use_cases.train_model:__init__:28 - TrainModelUseCase initialisé avec KerasCNNClassifier
2026-01-18 05:31:33.462 | INFO     | mh_nlp.application.use_cases.train_model:execute:50 - Démarrage de l'entraînement : 700 docs (train) | 100 docs (val)
2026-01-18 05:31:33.465 | INFO     | mh_nlp.infrastructure.nlp.keras_tokenizer:fit:48 - Apprentissage du vocabulaire sur 700 documents...
2026-01-18 05:31:33.480 | SUCCESS  | mh_nlp.infrastructure.nlp.keras_tokenizer:fit:54 - Vocabulaire appris : 1740 mots uniques trouvés.
2026-01-18 05:31:33.50

Entraînement du CNN Keras...
Epoch 1/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - accuracy: 0.6943 - loss: 0.7676 - val_accuracy: 0.7300 - val_loss: 0.5581
Epoch 2/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.7257 - loss: 0.5372 - val_accuracy: 0.7300 - val_loss: 0.4523
Epoch 3/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.8543 - loss: 0.3303 - val_accuracy: 0.8800 - val_loss: 0.2395
Epoch 4/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.9786 - loss: 0.1045 - val_accuracy: 0.9300 - val_loss: 0.1581
Epoch 5/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.9857 - loss: 0.0438 - val_accuracy: 0.9400 - val_loss: 0.1439
Epoch 6/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.9943 - loss: 0.0223 - val_accuracy: 0.9400 - val_loss: 0.1802
Epoch 7/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.9971 - loss: 0.0206 - val_accuracy: 0.9400 - val_loss: 0.1759
Epoch 8/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.9971 - loss: 0.00

2026-01-18 05:31:39.633 | SUCCESS  | mh_nlp.application.use_cases.train_model:execute:57 - Modèle entraîné avec succès via KerasCNNClassifier.
